<img src="https://courses.edx.org/asset-v1:ACCA+ML001+2T2021+type@asset+block@acca-logo.jpg" alt="ACCA logo" style="width: 400px;"/>

# Automating Excel using Python
## Part 3 - Generating multiple reports

* **Course:** __Machine learning with Python for finance professionals__ by ACCA
* **Instructor:** [Coefficient](https://coefficient.ai) / [@CoefficientData](https://twitter.com/CoefficientData)

---

In [ ]:
import pandas as pd
import xlwings as xw

# 

<div class="alert alert-block alert-info" style="background-color: #BA001E; border: 0px; -moz-border-radius: 10px; -webkit-border-radius: 10px;">
<h2 style="color: white">
Working with files in Python
</h2><br>
</div>

We will be working with Python's built-in module `pathlib`.

First, let's use it to create a directory/folder, if it doesn't exist yet.

In [ ]:
from pathlib import Path

In [ ]:
# path = 'C:\Users\data\'
# path = '/Users/me/data/'

In [ ]:
# The best reason to use pathlib's Path is that it can create "OS-native" filepaths, i.e. on
# Windows it will create paths separated by backslashes (\) and on macOS/Linux it will use
# forward slashes (/). Let's create a variable that points to the folder where this notebook lives.

here = Path(".")

In [ ]:
# Where are we?
here.resolve()

In [ ]:
# What's here? Let's list all the notebooks in this folder
for file in here.glob("*.ipynb"):
    print(file)

In [ ]:
# We'd like to create a folder in this location called "reports". Let's construct it
# step-by-step, starting with a Path that points to the (yet-to-be-created) reports folder.

reports_folder = (here / "reports")
reports_folder.resolve()

In [ ]:
# Unless you've run this notebook already, the reports folder won't exist. Let's create it now.
reports_folder.mkdir(exist_ok=True)

A directory called `reports` should have just appeared in the same directory as this notebook.

# 

<div class="alert alert-block alert-info" style="background-color: #BA001E; border: 0px; -moz-border-radius: 10px; -webkit-border-radius: 10px;">
<h2 style="color: white">
Demo: Generating multiple Excel reports with xlwings
</h2><br>
</div>

All we're going to do here is repeat the analysis from the previous notebook's solutions, but with a for loop that will generate one templated report for every vendor in the dataset.

You're already 90% there in terms of knowledge, but this is the real time saver!

### Read in the data

In [ ]:
# Read in the S-MartX data and replace missing values with an empty string
df = (
    pd.read_excel("S-MartX Database.xlsx", sheet_name="Main Database")
    .fillna("")
)

In [ ]:
# Get unique list of vendors
vendors = df["Vendor Name"].unique()
print(vendors)

### One function to get all the vendor stats

In [ ]:
# Refactor ALL the code from the last notebook into a single function.


def get_vendor_stats(df, vendor_name):
    # We will fill this dictionary with vendor stats and return it
    data = {}
    data["vendor"] = vendor_name

    # Filter to this vendor only
    vendor = df.query(f"`Vendor Name` == '{vendor_name}'")

    # Get order count
    data["order_count"] = len(vendor)

    # Get total net sales
    data["total_net_sales"] = vendor["Net Sales[SGD]"].sum()

    # Get top brand & top item in one go (less code is good!)
    for col in ["Brand Name", "Item Name"]:
        data[col] = (
            vendor.groupby([col])["Net Sales[SGD]"]
            .sum()
            .reset_index()
            .sort_values("Net Sales[SGD]", ascending=False)
            .iloc[0][col]
        )

    # Get top categories
    data["top_categories"] = (
        vendor.groupby(["Category Name", "Prod Grp Name"])["Net Sales[SGD]"]
        .sum()
        .reset_index()
        .sort_values("Net Sales[SGD]", ascending=False)
        .head(4)
        .values  # added this in here
    )

    # min/max dates
    data["min_date"] = vendor.Date.min().strftime("%d %b %Y")
    data["max_date"] = vendor.Date.max().strftime("%d %b %Y")

    return data

In [ ]:
# Let's take it for a spin
get_vendor_stats(df, vendor_name="Sports Hub")

### Let's generate some reports

In [ ]:
# We're going to iterate through the list of vendors created earlier
vendors[:3]

In [ ]:
# How many vendors are there?
len(vendors)

Let's use tqdm to give us a nice progress bar ([more info here](https://tqdm.github.io/)).

In [ ]:
from time import sleep
from tqdm import tqdm

In [ ]:
# tqdm generates nice progress bars!
for i in tqdm(range(20)):
    sleep(0.2)

In [ ]:
# If you'd like to process all 89 vendors, remove [:20] so it's just tqdm(vendors)
for vendor_name in tqdm(vendors[:20]):
    # Get our vendor stats
    vendor_stats = get_vendor_stats(df, vendor_name=vendor_name)

    # Open up the workbook and write it all in
    wb = xw.Book("S-MartX Vendor Report.xlsx")
    sheet = wb.sheets["Report"]
    sheet.range("B3").value = vendor_name
    sheet.range("C6").value = vendor_stats["min_date"]
    sheet.range("D6").value = vendor_stats["max_date"]
    sheet.range("D7").value = vendor_stats["order_count"]
    sheet.range("D8").value = vendor_stats["total_net_sales"]
    sheet.range("D9").value = vendor_stats["Brand Name"]
    sheet.range("D10").value = vendor_stats["Item Name"]
    sheet.range("F7").value = vendor_stats["top_categories"]

    # Specify where we want this saved
    filename = f"S-MartX Vendor Report - {vendor_name}.xlsx"
    filepath = reports_folder / filename

    # Save it
    wb.save(filepath)

If this ran successfully, check out the `reports/` folder, you should now see a bunch of tailored vendor reports. If you hit any errors, check to see if there are any dialogues asking for permissions (e.g. you will need to Grant Access to the `reports/` directory on macOS). If in doubt, try again.

Otherwise, read your error message carefully, Google the error message, see if there are any StackOverflow posts that might help. If you're very stuck, feel free to post your problem on StackOverflow and DM us [@CoefficientData](https://twitter.com/CoefficientData), perhaps we can help you out.

---
<div class="alert alert-block alert-success">
<b>🎉 Congratulations</b><br>
You have reached the end of this module.
</div>